In [1]:
import numpy as np
import os
import csv 
import torchvision
from PIL import Image
from torch.utils.data import Dataset
from torchvision import datasets, transforms
import torch
import torch.utils.data as datatorch
import torch.nn as nn
import torch.backends.cudnn as cudnn
#from google.colab import drive
import time
import datetime
import pandas as pd
import torch.nn.functional as F
from torchvision.models import vgg19
import math
from torchvision.utils import save_image, make_grid

In [2]:
labeled_data = pd.read_csv("cosmology_aux_data_170429/labeled.csv", dtype = np.int64).to_numpy()
scored_data = pd.read_csv("cosmology_aux_data_170429/scored.csv").to_numpy()

In [3]:
labeled_dir = 'cosmology_aux_data_170429/labeled'
query_dir = 'cosmology_aux_data_170429/query'
scored_dir = 'cosmology_aux_data_170429/scored'
labeled_files = os.listdir(labeled_dir)
scored_files = os.listdir(scored_dir)
query_files = os.listdir(query_dir)

In [4]:
params = {
    "bsize" : 32,# Batch size during training.
    'imsize' : 228,# Spatial size of training images. All images will be resized to this size during preprocessing.
    'nc' : 1,# Number of channles in the training images. For coloured images this is 3.
    'nz' : 100,# Size of the Z latent vector (the input to the generator).
    'ngf' : 128,# Size of feature maps in the generator. The depth will be multiples of this.
    'ndf' : 128, # Size of features maps in the discriminator. The depth will be multiples of this.
    'nepochs' : 2000,# Number of training epochs.
    'start_epoch': 0, #Epoch number to start from
    'lr' : 0.0002,# Learning rate for optimizers
    'beta1' : 0.5,# Beta1 hyperparam for Adam optimizer
    'save_epoch' : 100}# Save step.

In [5]:
class ImageDataset(Dataset):
    def __init__(self, file_array,dir, mode = 'train', labels = None, params = None):
        self.files = file_array
        self.mode = mode
        self.labels  = labels
        self.dir = dir
        self.img_size = params['imsize']
        # Transforms for low resolution images and high resolution images
        self.lr_transform = transforms.Compose(
            [
                transforms.Resize((self.img_size, self.img_size), Image.BICUBIC),
                #transforms.Grayscale( num_output_channels=1) ,
                transforms.ToTensor(),
                #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                transforms.Normalize([0.5], [0.5])
            ]
        )
        


    def __getitem__(self, index):
        img = Image.open(os.path.join(self.dir, self.files[index]))
        img_lr = self.lr_transform(img)
        #img_hr = self.hr_transform(img)
        
        if self.labels is not None:
            mask = np.isin(self.labels[:,0], int(self.files[index][:-4]))
            label = self.labels[mask,1]
            
            return img_lr,label
        else:
            return img_lr

    def __len__(self):
        return len(self.files)

In [6]:

scored_dataset = ImageDataset(scored_files, scored_dir, mode = 'train', labels =scored_data, params = params)
labeled_dataset = ImageDataset(labeled_files, labeled_dir, mode = 'train', labels =labeled_data, params = params)
query_dataset = ImageDataset(query_files, query_dir, mode = 'test', labels =None, params = params)

In [7]:
# Define the Generator Network
class Generator(nn.Module):
    def __init__(self, params):
        super().__init__()

        # Input is the latent vector Z.
        self.tconv1 = nn.ConvTranspose2d(params['nz'], params['ngf']*8,
            kernel_size=4, stride=1, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(params['ngf']*8)

        # Input Dimension: (ngf*8) x 4 x 4
        self.tconv2 = nn.ConvTranspose2d(params['ngf']*8, params['ngf']*4,
            4, 2, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(params['ngf']*4)

        # Input Dimension: (ngf*4) x 8 x 8
        self.tconv3 = nn.ConvTranspose2d(params['ngf']*4, params['ngf']*2,
            4, 2, 1, bias=False)
        self.bn3 = nn.BatchNorm2d(params['ngf']*2)

        # Input Dimension: (ngf*2) x 16 x 16
        self.tconv4 = nn.ConvTranspose2d(params['ngf']*2, params['ngf'],
            4, 2, 1, bias=False)
        self.bn4 = nn.BatchNorm2d(params['ngf'])

        # Input Dimension: (ngf) * 32 * 32
        self.tconv5 = nn.ConvTranspose2d(params['ngf'], params['nc'],
            4, 2, 1, bias=False)
        #Output Dimension: (nc) x 64 x 64

    def forward(self, x):
        x = F.relu(self.bn1(self.tconv1(x)))
        x = F.relu(self.bn2(self.tconv2(x)))
        x = F.relu(self.bn3(self.tconv3(x)))
        x = F.relu(self.bn4(self.tconv4(x)))

        x = torch.tanh(self.tconv5(x))

        return x

# Define the Discriminator Network
class Discriminator(nn.Module):
    def __init__(self, params):
        super().__init__()

        # Input Dimension: (nc) x 64 x 64
        self.conv1 = nn.Conv2d(params['nc'], params['ndf'],
            4, 2, 1, bias=False)

        # Input Dimension: (ndf) x 32 x 32
        self.conv2 = nn.Conv2d(params['ndf'], params['ndf']*2,
            4, 2, 1, bias=False)
        self.bn2 = nn.BatchNorm2d(params['ndf']*2)

        # Input Dimension: (ndf*2) x 16 x 16
        self.conv3 = nn.Conv2d(params['ndf']*2, params['ndf']*4,
            4, 2, 1, bias=False)
        self.bn3 = nn.BatchNorm2d(params['ndf']*4)

        # Input Dimension: (ndf*4) x 8 x 8
        self.conv4 = nn.Conv2d(params['ndf']*4, params['ndf']*8,
            4, 2, 1, bias=False)
        self.bn4 = nn.BatchNorm2d(params['ndf']*8)

        # Input Dimension: (ndf*8) x 4 x 4
        self.conv5 = nn.Conv2d(params['ndf']*8, 1, 4, 1, 0, bias=False)
        self.flatten1 = nn.Flatten()
    def forward(self, x):
        x = F.leaky_relu(self.conv1(x), 0.2, True)
        x = F.leaky_relu(self.bn2(self.conv2(x)), 0.2, True)
        x = F.leaky_relu(self.bn3(self.conv3(x)), 0.2, True)
        x = F.leaky_relu(self.bn4(self.conv4(x)), 0.2, True)

        #x = torch.sigmoid(self.conv5(x))
        x = torch.sigmoid(torch.sum(self.flatten1(self.conv5(x)),dim=1))
        #x = torch.sigmoid(self.flatten1(self.conv5(x)))
        return x

In [8]:
os.makedirs("saved_images", exist_ok=True)
os.makedirs("saved_models", exist_ok=True)

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Initialize generator and discriminator
netG = Generator(params).to(device)
netD = Discriminator(params).to(device)

# Losses
criterion = nn.BCELoss(reduction = 'sum')


# Optimizers
fixed_noise = torch.randn(1, params['nz'], 1, 1, device=device)
print(fixed_noise.size())
real_label = 1
fake_label = 0

# Optimizer for the discriminator.
optimizerD = torch.optim.Adam(netD.parameters(), lr=params['lr'], betas=(params['beta1'], 0.999))
# Optimizer for the generator.
optimizerG = torch.optim.Adam(netG.parameters(), lr=params['lr'], betas=(params['beta1'], 0.999))

# Stores generated images as training progresses.
img_list = []
# Stores generator losses during training.
G_losses = []
# Stores discriminator losses during training.
D_losses = []

iters = 0

labeled_Dataloader = datatorch.DataLoader(dataset=labeled_dataset, shuffle=False, batch_size=params['bsize'])
scored_Dataloader = datatorch.DataLoader(dataset=scored_dataset, shuffle=False,  batch_size=params['bsize'])

torch.Size([1, 100, 1, 1])


In [10]:
if params['start_epoch'] != 0:
    # Load pretrained models
    netG.load_state_dict(torch.load("saved_models/generator_%d.pth"))
    netD.load_state_dict(torch.load("saved_models/discriminator_%d.pth"))
    



In [11]:
netG.train()
netD.train()
for epoch in range(params['nepochs']):
    for i, (lr_img, label) in enumerate(labeled_Dataloader, 0):
        # Transfer data tensor to GPU/CPU (device)
        #label = label.to(device)
        lr_img = lr_img[label==1,:,:].unsqueeze(1).to(device)
        
        # Get batch size. Can be different from params['nbsize'] for last batch in epoch.
        b_size = lr_img.size(0)
        label = torch.full((b_size, ), real_label, device=device)
        
        # Make accumalated gradients of the discriminator zero.
        netD.zero_grad()
        # Create labels for the real data. (label=1)
        label = torch.full((b_size, ), real_label, device=device)
        
        output = netD(lr_img)
        
        errD_real = criterion(output, label)
        
        # Calculate gradients for backpropagation.
        errD_real.backward()
        D_x = output.mean().item()
        
        # Sample random data from a unit normal distribution.
        noise = torch.randn(b_size, params['nz'], 1, 1, device=device)
        # Generate fake data (images).
        fake_data = netG(noise)
        # Create labels for fake data. (label=0)
        label.fill_(fake_label)
        # Calculate the output of the discriminator of the fake data.
        # As no gradients w.r.t. the generator parameters are to be
        # calculated, detach() is used. Hence, only gradients w.r.t. the
        # discriminator parameters will be calculated.
        # This is done because the loss functions for the discriminator
        # and the generator are slightly different.
        
        output = netD(fake_data.detach()).view(-1)
        errD_fake = criterion(output, label)
        # Calculate gradients for backpropagation.
        
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        
        # Net discriminator loss.
        errD = errD_real + errD_fake
        # Update discriminator parameters.
        
        optimizerD.step()
        
        # Make accumalted gradients of the generator zero.
        netG.zero_grad()
        # We want the fake data to be classified as real. Hence
        # real_label are used. (label=1)
        label.fill_(real_label)
        # No detach() is used here as we want to calculate the gradients w.r.t.
        # the generator this time.
        output = netD(fake_data).view(-1)
        errG = criterion(output, label)
        
        # Gradients for backpropagation are calculated.
        # Gradients w.r.t. both the generator and the discriminator
        # parameters are calculated, however, the generator's optimizer
        # will only update the parameters of the generator. The discriminator
        # gradients will be set to zero in the next iteration by netD.zero_grad()
        errG.backward()

        D_G_z2 = output.mean().item()
        # Update generator parameters.
        optimizerG.step()
        
        # Check progress of training.
        if i%50 == 0:
            print(torch.cuda.is_available())
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, params['nepochs'], i, len(labeled_Dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        # Save the losses for plotting.
        G_losses.append(errG.item())
        D_losses.append(errD.item())
        # Check how the generator is doing by saving G's output on a fixed noise.
        if (iters % 100 == 0) or ((epoch == params['nepochs']-1) and (i == len(labeled_Dataloader)-1)):
            with torch.no_grad():
                fake_data = netG(fixed_noise).detach().cpu()
                gen_hr = make_grid(fake_data, nrow=1, normalize=True)
                save_image(gen_hr, "saved_images/%d.png" % iters, normalize=False)

        iters += 1

    # Save the model.
    if epoch % params['save_epoch'] == 1:
        torch.save({
            'generator' : netG.state_dict(),
            'discriminator' : netD.state_dict(),
            'optimizerG' : optimizerG.state_dict(),
            'optimizerD' : optimizerD.state_dict(),
            'params' : params
            }, 'saved_models/model_epoch_{}.pth'.format(epoch))
        
    
# Save the final trained model.
torch.save({
            'generator' : netG.state_dict(),
            'discriminator' : netD.state_dict(),
            'optimizerG' : optimizerG.state_dict(),
            'optimizerD' : optimizerD.state_dict(),
            'params' : params
            }, 'saved_models/model_final.pth')

True
[0/2000][0/38]	Loss_D: 30.6414	Loss_G: 50.7129	D(x): 0.8862	D(G(z)): 0.4904 / 0.1905
True
[1/2000][0/38]	Loss_D: 0.0426	Loss_G: 211.8797	D(x): 1.0000	D(G(z)): 0.0014 / 0.0009
True
[2/2000][0/38]	Loss_D: 0.0525	Loss_G: 215.1409	D(x): 1.0000	D(G(z)): 0.0017 / 0.0008
True
[3/2000][0/38]	Loss_D: 0.0163	Loss_G: 234.9102	D(x): 1.0000	D(G(z)): 0.0005 / 0.0004
True
[4/2000][0/38]	Loss_D: 0.0133	Loss_G: 240.0348	D(x): 1.0000	D(G(z)): 0.0004 / 0.0003
True
[5/2000][0/38]	Loss_D: 0.0319	Loss_G: 224.1647	D(x): 1.0000	D(G(z)): 0.0011 / 0.0006
True
[6/2000][0/38]	Loss_D: 0.0343	Loss_G: 227.3622	D(x): 1.0000	D(G(z)): 0.0011 / 0.0005
True
[7/2000][0/38]	Loss_D: 0.0282	Loss_G: 231.9897	D(x): 1.0000	D(G(z)): 0.0009 / 0.0005
True
[8/2000][0/38]	Loss_D: 0.0292	Loss_G: 234.6869	D(x): 1.0000	D(G(z)): 0.0010 / 0.0004
True
[9/2000][0/38]	Loss_D: 0.0173	Loss_G: 240.7975	D(x): 1.0000	D(G(z)): 0.0006 / 0.0004
True
[10/2000][0/38]	Loss_D: 0.0149	Loss_G: 243.9168	D(x): 1.0000	D(G(z)): 0.0005 / 0.0003
True
[11/

[90/2000][0/38]	Loss_D: 0.0001	Loss_G: 380.4805	D(x): 1.0000	D(G(z)): 0.0000 / 0.0000
True
[91/2000][0/38]	Loss_D: 0.0001	Loss_G: 387.1447	D(x): 1.0000	D(G(z)): 0.0000 / 0.0000
True
[92/2000][0/38]	Loss_D: 0.0001	Loss_G: 382.2955	D(x): 1.0000	D(G(z)): 0.0000 / 0.0000
True
[93/2000][0/38]	Loss_D: 0.0001	Loss_G: 378.9373	D(x): 1.0000	D(G(z)): 0.0000 / 0.0000
True
[94/2000][0/38]	Loss_D: 0.0001	Loss_G: 377.0182	D(x): 1.0000	D(G(z)): 0.0000 / 0.0000
True
[95/2000][0/38]	Loss_D: 0.0001	Loss_G: 380.1399	D(x): 1.0000	D(G(z)): 0.0000 / 0.0000
True
[96/2000][0/38]	Loss_D: 0.0001	Loss_G: 383.2737	D(x): 1.0000	D(G(z)): 0.0000 / 0.0000
True
[97/2000][0/38]	Loss_D: 0.0001	Loss_G: 382.9410	D(x): 1.0000	D(G(z)): 0.0000 / 0.0000
True
[98/2000][0/38]	Loss_D: 0.0001	Loss_G: 385.2004	D(x): 1.0000	D(G(z)): 0.0000 / 0.0000
True
[99/2000][0/38]	Loss_D: 0.0001	Loss_G: 388.6680	D(x): 1.0000	D(G(z)): 0.0000 / 0.0000
True
[100/2000][0/38]	Loss_D: 0.0001	Loss_G: 386.2534	D(x): 1.0000	D(G(z)): 0.0000 / 0.0000
Tru

OSError: broken data stream when reading image file

In [ ]:
print('finished')